In [0]:
use catalog uc_prod

### Getting the distinct states from the staing master table (silver.daily_pricing_silver)

In [0]:

create or replace table silver.dim_variety_staging_st1
select distinct variety from silver.daily_pricing_silver
where record_updated_date > (
      select nvl(max(process_file_updated_date),'2010-01-01') from uc_prod.watermark_tbl.process_logs 
      where process_name = 'reporting_dim_variety_gold' and process_status ='completed'
                            )


### Add the distinct unique id for the state items

In [0]:
create or replace table uc_prod.silver.dim_variety_gold_st_2
select s.variety, 
case when t.variety_id is null 
  then row_number() over(order by s.variety) 
  else t.variety_id end as variety_ID_st2
,t.variety_id as gold_variety_id
from uc_prod.silver.dim_variety_staging_st1 s
left join gold.reporting_dim_variety_gold t
on t.variety = s.variety
where t.variety is null or t.variety_id is not null

In [0]:
create or replace table uc_prod.silver.dim_variety_gold_st_3
select 
variety,
case when  src.gold_variety_id is null
then (src.variety_ID_st2+max_id) 
else  src.gold_variety_id end as VARIETY_ID,
current_timestamp() as lakehouse_inserted_date,
current_timestamp() as lakehouse_updated_date 
from uc_prod.silver.dim_variety_gold_st_2 as src
cross join (select nvl(max(VARIETY_ID),0) as max_id from uc_prod.gold.reporting_dim_variety_gold)

In [0]:
merge into gold.reporting_dim_variety_gold as tgt
using uc_prod.silver.dim_variety_gold_st_3 src
on tgt.variety = src.variety and src.variety is not null
when matched then update 
  set tgt.lakehouse_updated_date = current_timestamp()
when not matched then  
  insert (
      VARIETY_ID,
      VARIETY,
      lakehouse_inserted_date,
      lakehouse_updated_date
         )
  values
        (
        src.VARIETY_ID,
        src.VARIETY,
        src.lakehouse_inserted_date,
        src.lakehouse_updated_date
        )


### update the water mark table

In [0]:
insert into uc_prod.watermark_tbl.process_logs
(Process_name,Process_file_updated_date,process_status)
values
(
'reporting_dim_variety_gold',
current_timestamp(),
'completed'
)